In [98]:
import json
import os
from dataclasses import dataclass
import matplotlib.pyplot as plt
import tables
from dataforge import Meta, Name
from phd.thunderstorm.dwyer_feedback import get_bins_from_center, load_reversed_data, histogrammed_reverse_data, \
    histogrammed_reverse_data_v2, ReverseHistogramm, get_dwyer_line, FeedBack
from phd.utils.convertor_tools import direction_to_degree
from scipy.interpolate import UnivariateSpline
import numpy as np
import matplotlib as mpl
import tables
from phd.utils.path_tools import find_by_meta, get_attrs_values
from scipy.optimize import curve_fit
from tables import File, Group

In [167]:
path = "/mnt/storage2/phd/data/thunderstorm/dwyer2003/"
over_3mev = []
for i in range(6):
    path_hdf5 = os.path.join(path, f"diff_{i}.hdf5")
    with tables.open_file(path_hdf5) as h5file:
        for group in h5file.root:
            for name in ["gamma", "positron"]:
                table = h5file.get_node(group, name)
                number = table.attrs["values_macros_number"]
                height = table.attrs["values_gdml_height"]
                cell = table.attrs["values_gdml_cellHeight"]
                field = table.attrs["values_gdml_fieldValueZ"] 
                data = table.read()
                indx = data["energy"] > 3
                over_3mev.append([name, field, cell, indx.sum(), number, indx.sum()/number])

In [169]:
for item in over_3mev:
    if item[0] != "gamma": continue
    if item[1] == 0.001 or item[1]==0.0007: continue
    print(item[1], item[-1])


0.00055 1500.1333333333334
0.0006 45.0
0.00055 106.0
0.0005 48.0
0.00045 96.0
0.0004 0.0
0.0006 10.6
0.0006 13.2
0.0006 8.0
0.0006 4.0
0.0006 11.0
0.0006 11.0


In [170]:
for item in over_3mev:
    if item[0] != "positron": continue
    if item[1] == 0.001 or item[1]==0.0007: continue
    print(item[1], item[-1])



0.00055 72.46666666666667
0.0006 30.0
0.00055 67.0
0.0005 25.0
0.00045 72.0
0.0004 0.0
0.0006 10.3
0.0006 12.0
0.0006 8.2
0.0006 2.0
0.0006 8.2
0.0006 5.0
